In [ ]:
import ccxt
import pandas as pd
import time

# Initialize Bybit API with API keys
api_key = 'I6m01uNAWiensHbIBA'
api_secret = 'vkaviS9xw903NJPncMtHgGfyiWB4mnhSJTyK'
exchange = ccxt.bybit({
    'apiKey': api_key,
    'secret': api_secret,
    'enableRateLimit': True,
})

# Fetch trading fees dynamically from Bybit
def get_trading_fee():
    try:
        markets = exchange.load_markets()
        trading_fees = {symbol: markets[symbol]['taker'] for symbol in markets.keys()}
        return trading_fees
    except Exception as e:
        print("Error fetching trading fees:", str(e))
        return {}

trading_fees = get_trading_fee()

def fetch_all_prices():
    """Fetch real-time prices for all tokens available on Bybit."""
    tickers = exchange.fetch_tickers()
    prices = {symbol: data['last'] for symbol, data in tickers.items() if '/' in symbol}
    return prices

def find_arbitrage_opportunities(prices):
    """Identify profitable triangular arbitrage opportunities across all tokens."""
    opportunities = []
    
    for base in prices.keys():
        base_token = base.split('/')[0]
        for mid in prices.keys():
            mid_token = mid.split('/')[1]
            for quote in prices.keys():
                quote_token = quote.split('/')[1]
                if base_token != mid_token and mid_token != quote_token and base_token != quote_token:
                    pair1 = f"{base_token}/{mid_token}"
                    pair2 = f"{mid_token}/{quote_token}"
                    pair3 = f"{quote_token}/{base_token}"

                    if pair1 in prices and pair2 in prices and pair3 in prices:
                        rate1 = prices[pair1]
                        rate2 = prices[pair2]
                        rate3 = prices[pair3]

                        # Get trading fees dynamically for each pair
                        fee1 = trading_fees.get(pair1, 0.001)
                        fee2 = trading_fees.get(pair2, 0.001)
                        fee3 = trading_fees.get(pair3, 0.001)

                        # Calculate final amount after triangular trade
                        final_amount = (1 / rate1) * (1 / rate2) * (1 / rate3)

                        # Apply trading fees (3 trades with dynamic fees)
                        final_amount *= (1 - fee1) * (1 - fee2) * (1 - fee3)

                        profit_percent = (final_amount - 1) * 100

                        if profit_percent > 1:  # Ensure profit > 1%
                            opportunities.append({
                                'Base': base_token,
                                'Mid': mid_token,
                                'Quote': quote_token,
                                'Profit %': round(profit_percent, 3),
                                'Path': f"{base_token} → {mid_token} → {quote_token} → {base_token}"
                            })

    return opportunities

def execute_trade(pair, order_type="market", amount=10):
    """Execute a trade on Bybit."""
    try:
        print(f"Placing order: {order_type} {amount} of {pair}")
        order = exchange.create_market_order(pair, 'buy', amount)
        print("✅ Trade executed:", order)
    except Exception as e:
        print("❌ Trade failed:", str(e))

def main():
    """Main function: Scan for arbitrage and execute trades."""
    while True:
        try:
            print("\n🔄 Fetching latest prices...")
            prices = fetch_all_prices()
            opportunities = find_arbitrage_opportunities(prices)

            if opportunities:
                print("\n🔥 Profitable Arbitrage Found 🔥")
                df = pd.DataFrame(opportunities)
                print(df)

                # ✅ Auto-execute first profitable arbitrage trade
                best_trade = opportunities[0]
                base, mid, quote = best_trade['Base'], best_trade['Mid'], best_trade['Quote']
                
                print("\n🚀 Executing trade:", best_trade['Path'])
                
                # Execute the three trades (Market orders)
                execute_trade(f"{base}/{mid}")
                execute_trade(f"{mid}/{quote}")
                execute_trade(f"{quote}/{base}")
            else:
                print("\n❌ No profitable opportunities found.")

            time.sleep(5)  # Check every 5 seconds
        except Exception as e:
            print(f"⚠️ Error: {e}")
            time.sleep(5)

if __name__ == "__main__":
    main()



🔄 Fetching latest prices...

❌ No profitable opportunities found.

🔄 Fetching latest prices...

❌ No profitable opportunities found.

🔄 Fetching latest prices...

❌ No profitable opportunities found.

🔄 Fetching latest prices...

❌ No profitable opportunities found.

🔄 Fetching latest prices...

❌ No profitable opportunities found.

🔄 Fetching latest prices...

❌ No profitable opportunities found.

🔄 Fetching latest prices...

❌ No profitable opportunities found.

🔄 Fetching latest prices...

❌ No profitable opportunities found.

🔄 Fetching latest prices...

❌ No profitable opportunities found.

🔄 Fetching latest prices...

❌ No profitable opportunities found.

🔄 Fetching latest prices...

❌ No profitable opportunities found.

🔄 Fetching latest prices...

❌ No profitable opportunities found.

🔄 Fetching latest prices...

❌ No profitable opportunities found.

🔄 Fetching latest prices...

❌ No profitable opportunities found.

🔄 Fetching latest prices...

❌ No profitable opportunities fo